# Sklearn RBM

In [1]:
!mkdir llh_fig
!mkdir weights
!mkdir lr_fig

mkdir: cannot create directory ‘llh_fig’: File exists
mkdir: cannot create directory ‘weights’: File exists
mkdir: cannot create directory ‘lr_fig’: File exists


# RBM Class

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import Binarizer
import tensorflow as tf
from scipy.special import expit

def Hinton_bias_init(x, x_min=0, level_gap=1):
    '''
    Initialize visible biases based on the average value in the dataset.
    Hinton, "A Practical Guide to Training Restricted Boltzmann Machines"
    x: data array
    x_min: minimum value of x
    level_gap: difference between max and min values in x
    '''
    xmean = np.mean(x, axis=0)
    S = 1e-4
    x1, x2 = x_min + S, 1 - S
    xmean = np.clip(xmean, x1, x2)
    return (1 / level_gap) * np.clip(np.log(xmean - x_min) - np.log(1 - xmean), -300, 300)

class RBM:
    def __init__(self, n_visible, n_hidden, initial_learning_rate=0.01, decay_rate=0.95, batch_size=10, n_iter=20, optimizer='sgd', cd_steps=1, spins=False, potts=False, l1_reg=0.0, init_method='glorot', n_jobs=-1, random_state=None):
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.initial_learning_rate = initial_learning_rate
        self.learning_rate = initial_learning_rate
        self.decay_rate = decay_rate
        self.batch_size = batch_size
        self.n_iter = n_iter
        self.optimizer = optimizer.lower()
        self.cd_steps = cd_steps
        self.spins = spins
        self.potts = potts
        self.l1_reg = l1_reg
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.weights = self._initialize_weights(init_method)
        self.visible_bias = np.zeros(n_visible)
        self.hidden_bias = np.zeros(n_hidden)
        self.opt = self._choose_optimizer()
        self.learning_rate_history = []
        self.log_likelihood_history = []
        # Setup for spins/Potts/other configurations
        if self.spins:
            self.x_min = -1
            self.level_gap = 2.0
        else:
            self.x_min = 0
            self.level_gap = 1.0
        
        if self.potts:
            self.str_simul = "RBM_Potts"
            self.nz = self.n_hidden
        else:
            self.str_simul = "RBM"
            self.nz = 2 ** self.n_hidden
        
        if self.potts and self.spins:
            raise ValueError("POTTS and SPINS cannot coexist")

        # Set random seed if provided
        if self.random_state is not None:
            np.random.seed(self.random_state)
            tf.random.set_seed(self.random_state)

    def _initialize_weights(self, method):
        if method == 'xavier':
            scale = np.sqrt(2.0 / (self.n_visible + self.n_hidden))
        elif method == 'he':
            scale = np.sqrt(2.0 / self.n_visible)
        elif method == 'glorot':  ### GLOROT is default
            scale = np.sqrt(4. / float(self.n_visible + self.n_hidden))
        return np.random.randn(self.n_visible, self.n_hidden) * scale

    def _choose_optimizer(self):
        if self.optimizer == 'sgd':
            self.initial_learning_rate = 1.0
            self.decay_rate = 0.95
            return tf.optimizers.SGD(self.learning_rate)
        elif self.optimizer == 'rmsprop':
            self.initial_learning_rate = 0.05
            self.decay_rate = 1.0
            return tf.optimizers.RMSprop(self.learning_rate)
        elif self.optimizer == 'adam':
            self.initial_learning_rate = 0.001
            self.decay_rate = 1.0
            return tf.optimizers.Adam(self.learning_rate)
        else:
            raise ValueError("Unsupported optimizer. Choose from 'sgd', 'rmsprop', 'adam'.")

    def _update_learning_rate(self, epoch):
        self.learning_rate = self.initial_learning_rate * (self.decay_rate ** epoch)
        self.opt.learning_rate.assign(self.learning_rate)

    def _sigmoid(self, x):
        return expit(x)


    def _sample_prob(self, prob):
        return np.random.binomial(1, prob)

    def _l1_regularization(self):
        return self.l1_reg * np.sum(np.abs(self.weights))

    def contrastive_divergence(self, input_data):
        positive_hidden_probs = np.nan_to_num(self._sigmoid(np.dot(input_data, self.weights) + self.hidden_bias), 0.0)
        positive_hidden_activations = self._sample_prob(positive_hidden_probs)

        for step in range(self.cd_steps):
            negative_visible_probs = np.nan_to_num(self._sigmoid(np.dot(positive_hidden_activations, self.weights.T) + self.visible_bias), 0.0)
            negative_visible_activations = self._sample_prob(negative_visible_probs)
            negative_hidden_probs = np.nan_to_num(self._sigmoid(np.dot(negative_visible_activations, self.weights) + self.hidden_bias),0.0)
            positive_hidden_activations = self._sample_prob(negative_hidden_probs)

        return input_data, positive_hidden_probs, positive_hidden_activations, negative_visible_probs, negative_hidden_probs

    def fit(self, data):
        # Initialize visible biases using Hinton's method
        self.visible_bias = Hinton_bias_init(data, x_min=self.x_min, level_gap=self.level_gap)
        for epoch in range(self.n_iter+1):
            np.random.shuffle(data)
            batches = [data[k:k + self.batch_size] for k in range(0, data.shape[0], self.batch_size)]
            
            # Update learning rate
            self._update_learning_rate(epoch)
            self.learning_rate_history.append(self.learning_rate)

            for batch in batches:
                input_data, positive_hidden_probs, positive_hidden_activations, negative_visible_probs, negative_hidden_probs = self.contrastive_divergence(batch)
                
                positive_grad = np.dot(input_data.T, positive_hidden_probs)
                negative_grad = np.dot(negative_visible_probs.T, negative_hidden_probs)
                
                self.weights += self.learning_rate * (positive_grad - negative_grad) / self.batch_size
                self.visible_bias += self.learning_rate * np.mean(input_data - negative_visible_probs, axis=0)
                self.hidden_bias += self.learning_rate * np.mean(positive_hidden_probs - negative_hidden_probs, axis=0)

                # Apply L2 and L1 regularization
                self.weights -=  self._l1_regularization()

            if epoch % 5 == 0:
                log_likelihood = self.compute_log_likelihood(data)
                print(f"Epoch {epoch + 1}/{self.n_iter}, Log-Likelihood: {log_likelihood:.2f}, lr = {self.opt.learning_rate.numpy():.2f}")
                self.log_likelihood_history.append(log_likelihood)

    def transform(self, data):
        hidden_probs = self._sigmoid(np.dot(data, self.weights) + self.hidden_bias)
        return hidden_probs

    def compute_log_likelihood(self, data):
        visible_bias_term = np.dot(data, self.visible_bias)
        hidden_bias_term = np.sum(np.log(1 + np.exp(np.clip(np.dot(data, self.weights) + self.hidden_bias, -700,+680))), axis=1)
        log_likelihood = np.mean(visible_bias_term + hidden_bias_term)
        return np.nan_to_num(log_likelihood,7)

    def plot_learning_rate(self):
        plt.plot(self.learning_rate_history)
        plt.title("Learning Rate Over Time")
        plt.xlabel("Epoch")
        plt.ylabel("Learning Rate")
        plt.show()

    def plot_log_likelihood(self):
        with open(f'./llh_fig/llh_hist_L{self.n_hidden}_CD{self.cd_steps}_nepoch{self.n_iter}_grad{self.optimizer}.txt', "w") as file:
            for item in self.log_likelihood_history:
                file.write(str(item) + "\n")
        plt.plot(self.log_likelihood_history)
        plt.title("Log-Likelihood Over Time")
        plt.xlabel("Epoch (every 5th epoch)")
        plt.ylabel("Log-Likelihood")
        
        plt.savefig(f'./llh_fig/llh_hist_L{self.n_hidden}_CD{self.cd_steps}_nepoch{self.n_iter}_grad{self.optimizer}.png')
        plt.show()

    def cross_validate_with_seeds(self, data, seeds):
        log_likelihoods = []
        for seed in seeds:
            # Set the random seed
            np.random.seed(seed)
            tf.random.set_seed(seed)
            # Reinitialize the model with the same parameters but a different random seed
            rbm = RBM(n_visible=self.n_visible, n_hidden=self.n_hidden, initial_learning_rate=self.initial_learning_rate,
                      decay_rate=self.decay_rate, batch_size=self.batch_size, n_iter=self.n_iter, optimizer=self.optimizer,
                      cd_steps=self.cd_steps, spins=self.spins, potts=self.potts, l1_reg=self.l1_reg,
                      init_method='glorot', random_state=seed)
            # Fit the model to the data
            rbm.fit(data)
            # Compute the log-likelihood
            log_likelihood = rbm.compute_log_likelihood(data)
            log_likelihoods.append(log_likelihood)
            print(f"Seed {seed}, Log-Likelihood: {log_likelihood:.2f}")
        mean_log_likelihood = np.mean(log_likelihoods)
        std_log_likelihood = np.std(log_likelihoods)
        print(f'Mean Log-Likelihood: {mean_log_likelihood:.2f}')
        print(f'Standard Deviation of Log-Likelihood: {std_log_likelihood:.2f}')
        return mean_log_likelihood, std_log_likelihood

    def sample(self, n_samples, n_gibbs_steps=1000):
        samples = np.random.rand(n_samples, self.n_visible) > 0.5
        
        for step in range(n_gibbs_steps):
            hidden_probs = self._sigmoid(np.dot(samples, self.weights) + self.hidden_bias)
            hidden_activations = self._sample_prob(hidden_probs)
            visible_probs = self._sigmoid(np.dot(hidden_activations, self.weights.T) + self.visible_bias)
            samples = self._sample_prob(visible_probs)
        
        return samples

    def plot_samples(self, samples, n_cols=10):
        n_samples, n_features = samples.shape
        n_rows = int(np.ceil(n_samples / n_cols))
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols, n_rows))
        for i, ax in enumerate(axes.flat):
            if i < n_samples:
                ax.imshow(samples[i].reshape(int(np.sqrt(n_features)), int(np.sqrt(n_features))), cmap="gray")
            ax.axis('off')
        plt.show()

        
def plot_weights_bias(rbm, epoch, L, 
                      side=0, cols=0, thr=0, s=1.5, 
                      title=False, save=True, cmap="bwr"):
    '''
    Plot the weights of the RBM, one plot for each hidden unit.
    '''
    rows = int(np.ceil(L / cols))
    if rows == 1: rows = 2
    w = rbm.weights
    b = rbm.visible_bias
    if side == 0: side = int(np.sqrt(len(w)))
    if thr == 0: thr = 4
    plt.clf()
    fig, AX = plt.subplots(rows, cols + 1, figsize=(s * (1 + cols), s * rows))
    if title: fig.suptitle(f"epoch = {epoch}")
    k = 1
    for i in range(rows):
        for j in range(cols):
            if rows == 1: ax = AX[j + 1]
            else: ax = AX[i, j + 1]
            if k <= L:
                ax.imshow(w[:, k - 1].reshape(side, side), cmap=cmap, vmin=-thr, vmax=thr)
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_title(f"hidden {k}")
            else: fig.delaxes(ax)
            k += 1
        if i > 0:  fig.delaxes(AX[i, 0])
    
    ax = AX[0, 0]
    im = ax.imshow(b.reshape(side, side), cmap=cmap, vmin=-thr, vmax=thr)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title("bias")
    # colobar
    cbar_ax = fig.add_axes([0.14, 0.15, 0.024, 0.33])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.ax.tick_params(labelsize=12)
    
    S = 0.3
    plt.subplots_adjust(hspace=S)

    if save: plt.savefig(f"./FIG/FRAME/RBM_{epoch}_w-a.png")

    plt.show()
    plt.close()

In [37]:
# Load data
X_original, Y_original = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Parameters
Ndigit = 3
L = 3
optimizer = 'sgd'

# Select data
list_10_digits = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
list_digits = list_10_digits[-Ndigit:]
keep = np.isin(Y_original, list_digits)
X_keep, Y = X_original[keep], Y_original[keep]

# Binarize data
binarizer = Binarizer(threshold=127.5)
data = binarizer.fit_transform(X_keep)

# Initialize and train the RBM
rbm = RBM(n_visible=data.shape[1], n_hidden=L, initial_learning_rate=1., decay_rate=0.95, batch_size=10, n_iter=20, optimizer=optimizer, cd_steps=1, spins=False, potts=False, l1_reg=0.001, init_method='glorot')
rbm.fit(data)

/home/gigi/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Epoch 1/20, Log-Likelihood: 0.00, lr = 1.00
Epoch 6/20, Log-Likelihood: 0.00, lr = 0.77
Epoch 11/20, Log-Likelihood: 0.00, lr = 0.60
Epoch 16/20, Log-Likelihood: 0.00, lr = 0.46
Epoch 21/20, Log-Likelihood: 0.00, lr = 0.36


In [ ]:
# Perform cross-validation with different random seeds
seeds = [1, 2, 3, 4, 5]
#mean_log_likelihood, std_log_likelihood = rbm.cross_validate_with_seeds(data, seeds)

# Generate new samples
n_samples = 10
samples = rbm.sample(n_samples, n_gibbs_steps=100)

# Plot the generated samples
rbm.plot_samples(samples, n_cols=10)

# Plot the learning rate
rbm.plot_learning_rate()

# Plot the log-likelihood
rbm.plot_log_likelihood()

# Example usage
plot_weights_bias(rbm, epoch=10, L=3, cols=3, title=True)

/home/gigi/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Epoch 1/20, Log-Likelihood: nan, lr = 1.00
Epoch 6/20, Log-Likelihood: nan, lr = 0.77
Epoch 11/20, Log-Likelihood: nan, lr = 0.60
Epoch 16/20, Log-Likelihood: nan, lr = 0.46


ValueError: p < 0, p > 1 or p contains NaNs